### Functions to translate sequence encodings to start time encodings

In [5]:
# creates a map containing job operations according to job operation assignment and the duration according to the assignment vector
def get_sorted_duration_map(job_operation_assignment, durations):
    sorted_result_map = {}
    operation_count = {}

    for i, job in enumerate(job_operation_assignment):
        if job not in operation_count:
            operation_count[job] = 0
        else:
            operation_count[job] += 1

        operation = operation_count[job]
        sorted_result_map[(job, operation)] = durations[i]

    return sorted_result_map

# creates an object which contains neccessary information for determining start times - maps job operation, machine and duration 
def create_info_map(job_sequence, sorted_result_map, machine_assignment):
    occurrences = {}
    info = []

    for i, job in enumerate(job_sequence):
        if job not in occurrences:
            occurrences[job] = 0
        else:
            occurrences[job] += 1

        operation = occurrences[job]
        machine = machine_assignment[i]
        duration = sorted_result_map[(job, operation)][machine]

        info.append(((job, operation), machine, duration))
    
    return info

# takes start times in order of the sequence vector and sorts them according to the job operation assignment
def sort_start_times(sorted_result_map, start_times):
    sorted_start_times = []
    for job_op in sorted_result_map:
        for start_time in start_times:
            if start_time[0] == job_op:
                sorted_start_times.append(start_time[1])
                break
    return sorted_start_times

# determines the start time vector for a given job sequence vector, machine assignment vector and duration matrix
def sequence_to_starttime(job_operation_assignment, job_sequence, machine_assignment, durations):
    sorted_result_map = get_sorted_duration_map(job_operation_assignment, durations)
    info = create_info_map(job_sequence, sorted_result_map, machine_assignment)

    job_progress = [0] * len(set(job_sequence))
    machine_availability = [0] * len(set(machine_assignment))
    start_times = []

    for operation in info:
        job, job_operation = operation[0]
        duration = operation[2]
        machine = operation[1]

        start = max(machine_availability[machine], job_progress[job])

        job_progress[job] = start + duration
        machine_availability[machine] = start + duration
        start_times.append(((job, job_operation), start))

    return sort_start_times(sorted_result_map, start_times)

# example
job_sequence = [0, 1, 1, 2, 0, 2, 1]
machine_assignment = [1, 1, 0, 0, 1, 2, 2]
durations = [[10,5,0], [5,10,0], [0,30,0], [20,45,30], [0,2,10], [15,0,20], [0,0,70]]
job_operation_assignment = sorted(job_sequence)

print(sequence_to_starttime(job_operation_assignment, job_sequence, machine_assignment, durations))  # Output: [0, 35, 5, 35, 140, 55, 70]


[0, 35, 5, 35, 140, 55, 70]
